In [1]:
import numpy as np
import random
import itertools
import math
import tqdm
from tqdm import trange
import time

In [2]:
np.log(0.000000000000000)

<ipython-input-2-795782b68787>:1: RuntimeWarning: divide by zero encountered in log
  np.log(0.000000000000000)


-inf

# Helper Function

In [3]:
key_mapping={
    'C':0,
    'D':2,
    'E':4,
    'F':5,
    'G':7,
    'A':9,
    'B':11
}
def key2num(key):  
  key=key.upper()
  num=key_mapping[key[0]]
  modifier=len(key)
  if modifier==1:
    return num
  elif key[1]=='#':
    return (num+(modifier-1))%12
  elif key[1]=='B' or key[1]=='-':
    return (num-(modifier-1))%12
  elif key[1]=='X':
    return (num+(modifier-1)*2)%12

# key_list to number_list
def keys2num(keys):
    return [key2num(key) for key in keys]

In [4]:
# function to distingish whether notes in given timestamp and chord are within or outside the chord (0=outside,1=within)
import json
with open('../modules/json_files/keychorddict.json') as json_file:
    chord_notes = json.load(json_file)
  
    def note_2_class(chord,notes_at_t,chord_notes=chord_notes):
        notes_at_t=keys2num(notes_at_t.keys())
        note_in_chord=chord_notes[chord]['idx']
        return [int(note in note_in_chord) for note in notes_at_t]

# HMM

In [5]:


class HMM:
    def __init__(self,no_of_state,no_of_value,states,values):
        #randomize all matrix, each row should sum to 1
        #self.emssion_matrix=np.array([ran/ran.sum() for ran in np.array([np.random.rand(no_of_value) for i in range(no_of_state)])]) #b[i][O]  --> probability to emit values[O] from states[i]
        

        self.zero=0.00000000001
        self.emssion_matrix=np.random.rand(no_of_value)
        self.emssion_matrix=self.emssion_matrix/self.emssion_matrix.sum()
        if self.emssion_matrix[0]>self.emssion_matrix[1]:
            temp=self.emssion_matrix[0]
            self.emssion_matrix[0]=self.emssion_matrix[1]
            self.emssion_matrix[1]=temp
        self.emssion_matrix=np.array([self.emssion_matrix,]*len(states)) 
        self.emssion_matrix=np.array([[0.2,0.8],]*len(states)) 
        # assume same probability to emit "note within chord"  for all chords
        
        self.initial_matrix= np.random.rand(no_of_state) #π[i] --> probability to start at states[i]
        self.initial_matrix/= self.initial_matrix.sum()
        
        self.transition_matrix=np.array([ran/ran.sum() for ran in np.array([np.random.rand(no_of_state) for i in range(no_of_state)])])  #a[i][j] --> probability from states[i] transit to states[j]
        self.T1=None
        self.T2=None
        self.no_of_state=no_of_state
        self.states=states
        self.values=values
        self.observered=None
        self.key=None
 
        self.MinorMask=np.array([1, 1,
          1, 1, 1,
          1,
          1, 1,
          1, 1, 1,
          1, 1,1,1,
          1, 1,
          self.zero,
          self.zero,self.zero,self.zero,
          self.zero,
          self.zero,
          self.zero,self.zero,
          self.zero,self.zero,self.zero,self.zero,self.zero,
          self.zero,self.zero,self.zero
        ])
        self.MajorMask=(self.MinorMask*-1)+1+self.zero
        
        self.probit_at_i_table=None
        self.probit_transit_i_j_table=None
        self.forward_table=None
        self.backward_table=None
        self.chord_probit=None
        
    def debug(self):
        print('initial_matrix\n',self.initial_matrix)
        print('transition_matrix\n',self.transition_matrix)
        print('emission_matrix\n',self.emssion_matrix)
    
    def likelihood(self,state,ob_t,key):
        if key[-5:]!=self.states[state][:5]:
            return np.log(self.zero)
        chord=key+self.states[state][5:]

        a=noteToChord.NoteToChord(ob_t,key,3,0)
        score = np.array([i['Score'] for i in a])
        score=score-min(score)
        score/=score.sum()
        chords=[i['Chord'] for i in a]
      
        try:
            idx=chords.index(chord)
        except ValueError:
            idx=None
            
        if idx==None or score[idx]==0:
            return np.log(self.zero)

        ob_t=note_2_class(chord,ob_t) #2 class
        
        prob=1
        for x in ob_t:
            prob*=(self.emssion_matrix[state][x])
        return np.log(score[idx])+np.log(prob)#/self.chord_probit[state] # assume uniform probability

    def forward(self,t,j,ob=None,mode=False):

        if ob is None:
            ob=self.observered
        if np.isnan(self.forward_table[t][j]).any():
            if t==0:
                if mode==True:
                    if 'Major' in self.key[t]:
                        self.forward_table[t][j]=[(np.log(self.initial_matrix)+np.log(self.MajorMask))[j]+self.likelihood(j,ob[t],self.key[t]),0]
                    else:
                        self.forward_table[t][j]=[(np.log(self.initial_matrix)+np.log(self.MinorMask))[j]+self.likelihood(j,ob[t],self.key[t]),0]
                    return self.forward_table[t][j][0],self.forward_table[t][j][1]
                else:
                    self.forward_table[t][j]=self.initial_matrix[j]*self.likelihood(j,ob[t])
                    return self.forward_table[t][j]
            else:          
                if mode==True:

                    if self.key[t-1]==self.key[t]:
            
                        if 'Major' in self.key[t]:                            
                            result=np.array([self.forward(t-1,i,ob,mode)[0]+np.log(self.transition_matrix[i][j]) for i in range(self.no_of_state)])+np.log(self.MajorMask)+self.likelihood(j,ob[t],self.key[t])
                        else:
                            result=np.array([self.forward(t-1,i,ob,mode)[0]+np.log(self.transition_matrix[i][j]) for i in range(self.no_of_state)])+np.log(self.MinorMask)+self.likelihood(j,ob[t],self.key[t])

                    else:#if key change, dont consider chord transition, in other word, the chord before and after key change are consider as indepent
                        #if 'Major' in self.key[t]:                            
                         #   result=np.array([self.forward(t-1,i,ob,mode)[0] for i in range(self.no_of_state)])* self.likelihood(j,ob[t],self.key[t])
                        #else:
                            result=np.array([self.forward(t-1,i,ob,mode)[0] for i in range(self.no_of_state)])+self.likelihood(j,ob[t],self.key[t])
    
        
                    self.forward_table[t][j]=[np.max(result),np.argmax(result)]
                    return self.forward_table[t][j][0],self.forward_table[t][j][1]
                else:
                    self.forward_table[t][j]=sum([self.forward(t-1,i,ob,mode)*self.transition_matrix[i][j] for i in range(self.no_of_state)]) * self.likelihood(j,ob[t])
                    return self.forward_table[t][j]
        else:
            return self.forward_table[t][j]

    def backward(self,t,i,ob=None,mode=False):
        if ob is None:
            ob=self.observered
        if np.isnan(self.backward_table[t][i]) or mode==True:
            if t==len(ob)-1:
                self.backward_table[t][i]=1
                return self.backward_table[t][i]
            else:
                if mode==True:
                    self.backward_table[t][i]=max([self.transition_matrix[i][j]*self.likelihood(j,ob[t+1])*self.backward(t+1,j,ob,mode) for j in range(self.no_of_state)])
                    return self.backward_table[t][i]
                else:
                    self.backward_table[t][i]=sum([self.transition_matrix[i][j]*self.likelihood(j,ob[t+1])*self.backward(t+1,j,ob,mode) for j in range(self.no_of_state)])
                    return self.backward_table[t][i]
        else:
            return self.backward_table[t][i]
            
           
    def probit_at_i(self,t,i,ob=None):#Gamma γt(i) = P(qt = i|O,λ)      
        if ob is None:
            ob=self.observered
        if np.isnan(self.probit_at_i_table[t][i]):
            numerator=self.forward(t,i,ob)*self.backward(t,i,ob)#sum probability of all path passing through state[i] at time t
            denominator=sum([self.forward(t,j,ob)*self.backward(t,j,ob) for j in range(self.no_of_state)]) #prob of passing through  ALL_state at time t
            self.probit_at_i_table[t][i]=numerator/denominator
            return self.probit_at_i_table[t][i]
        else:
            return self.probit_at_i_table[t][i]
    
    def probit_transit_i_j(self,t,i,j,ob=None):#epsilon ξt(i, j) = P(qt = i,qt+1 = j|O,λ)
        if ob is None:
            ob=self.observered
        if np.isnan(self.probit_transit_i_j_table[t][i][j]):
            numerator=self.forward(t,i,ob)*self.transition_matrix[i][j]*self.likelihood(j,ob[t+1])*self.backward(t+1,j,ob)#sum probability of all path transit from state[i] to state[j] at time t
            denominator=sum([sum([self.forward(t,m,ob)*self.transition_matrix[m][n]*self.likelihood(n,ob[t+1])*self.backward(t+1,n,ob) for n in range(self.no_of_state)]) for m in range(self.no_of_state)]) #prob of ALL transition combination at time t
            self.probit_transit_i_j_table[t][i][j]=(numerator/denominator)
            return self.probit_transit_i_j_table[t][i][j]
        else:
            return self.probit_transit_i_j_table[t][i][j]
        
    #modify from https://stackoverflow.com/questions/9729968/python-implementation-of-viterbi-algorithm/9730083 , author RBF06(https://stackoverflow.com/users/3311728/rbf06)
    def predict(self,ob,test_key):  
        self.forward_table=np.empty((len(ob),self.no_of_state,2))
        self.forward_table[:]= np.NaN
        self.key=test_key

        T1=np.empty((self.no_of_state,len(ob)),'d')
        T2=np.empty((self.no_of_state,len(ob)),'B')
        for idx in range(self.no_of_state):
            T1[idx,0]=self.forward(0,idx,ob,True)[0]
        T2[:,0]=0
        self.T1=T1
        
        for i in range(1,len(ob)):
            for idx in range(self.no_of_state):
                T1[idx,i],T2[idx,i]=self.forward(i,idx,ob,True)

        x = np.empty(len(ob), 'B')
        x[-1] = np.argmax(T1[:, len(ob) - 1])
        
        self.T2=x
        for i in reversed(range(1, len(ob))):
            x[i - 1] = T2[x[i], i]
        return x
    
    def get_detail(self,label):
        key='Major' if 'M' in label else 'Minor'
        chord=label.split("_")[1]
        key_note=label.split('M')[0] if key=='Major' else label.split('m')[0].lower()
        if key=='Major':
            if key_note=='F#':
                key_note='Gb'
            elif key_note=='A#':
                key_note='Bb'
            elif key_note=='C#':
                key_note='Db'
            elif key_note=='G#':
                key_note='Ab'
            elif key_note=='D#':
                key_note='Eb'
        else:
            if key_note=='gb':
                key_note='f#'
            elif key_note=='db':
                key_note='c#'
            elif key_note=='ab':
                key_note='g#'
            elif key_note=='d#':
                key_note='eb'
                
        if 'German' in chord:
            chord=key+'GerVI'
        elif 'Dim7' in chord:
            chord=key+'DimVII7'
        elif 'FrenchVI' in chord:
            chord=key+'FreVI'
        else:
            chord=key+chord
        return chord,key_note
    
    def train_supervisied(self,obs,labels): # by MLE
        initial_matrix= np.zeros(self.no_of_state)
        emssion_matrix=np.zeros((2)) 
        transition_matrix=np.array([np.zeros(self.no_of_state) for i in range(self.no_of_state)])

        for idx_1,label in enumerate(labels):
            for idx_2,lab in enumerate(label):
                Pre_chord,Pre_key_name=self.get_detail(label[idx_2-1])
                chord,key_name=self.get_detail(label[idx_2])
                if idx_2==0:
                    initial_matrix[self.states.index(chord)]+=1
                else:
                    if Pre_chord[:5]==chord[:5]: ## ignore key change
                        transition_matrix[self.states.index(Pre_chord)][self.states.index(chord)]+=1
                        #note_2_class(chord,notes_at_t,chord_notes=chord_notes):
                    ob_t=note_2_class(key_name+chord,obs[idx_1][idx_2]) #2 class
                    for x in ob_t:
                        emssion_matrix[x]+=1
                        
        #save back to model
        self.initial_matrix=np.array([item/initial_matrix.sum() if item >0 else 0.00000001 for item in initial_matrix])
        self.transition_matrix=np.array([row/row.sum() if row.sum()>0 else row+0.00000001 for row in transition_matrix])
        for row in range(self.transition_matrix.shape[0]):
            for col in range(self.transition_matrix.shape[1]):
                if self.transition_matrix[row][col]==0:
                    self.transition_matrix[row][col]=0.00000001
        self.emssion_matrix=np.array([emssion_matrix/emssion_matrix.sum()] *self.no_of_state) 

        
"""        
    def train(self,obs,key_name,epochs=2):
        #O:observed values
        #λ:model parameters

        
        for epoch in range(epochs):
            for ob_idx,ob in enumerate(obs):
                self.update_state_name([key_name[ob_idx].lower()[:-1]+states for states in h_states])
                print('running',ob_idx,key_name[ob_idx].lower()[:-1])
                self.probit_at_i_table=np.empty((len(ob),self.no_of_state))
                self.probit_transit_i_j_table=np.empty((len(ob),self.no_of_state,self.no_of_state))
                self.forward_table=np.empty((len(ob),self.no_of_state))
                self.backward_table=np.empty((len(ob),self.no_of_state))
                self.probit_at_i_table[:]= np.NaN
                self.probit_transit_i_j_table[:]=np.NaN
                self.forward_table[:]= np.NaN
                self.backward_table[:]=np.NaN
                self.chord_probit=self.transition_matrix.sum(axis=0)
                self.chord_probit/=self.chord_probit.sum()
                #initalize DP table
                for t, i in tqdm.tqdm(itertools.product(range(len(ob)),range(self.no_of_state))):
                    self.forward(t,i,ob)
                    self.backward(t,i,ob)
                    self.probit_at_i(t,i,ob)
                    for j in range(self.no_of_state):
                        if t!=len(ob)-1:
                            self.probit_transit_i_j(t,i,j,ob)
                
                
                
                #initial matrix
                for i in range(self.no_of_state):
                    self.initial_matrix[i]=self.probit_at_i(0,i,ob)
                    if self.initial_matrix[i]==0:
                        self.initial_matrix[i]=0.00000001
                #transition matrix
                for i, j in itertools.product(range(self.no_of_state),range(self.no_of_state)):
                    self.transition_matrix[i][j]=sum([self.probit_transit_i_j(t,i,j,ob) for t in range(len(ob)-1)])/sum([self.probit_at_i(t,i,ob) for t in range(len(ob)-1)])
                    if self.transition_matrix[i][j]==0 :
                        self.transition_matrix[i][j]=0.00000001
                #emission matrix
                for j, k in itertools.product(range(self.no_of_state),range(len(self.values))):   
                    total=0
                    
                    #Modification: convert notes to 2 classes (outside or inside given chord)
                    chord=self.states[j]  #numeric to chord name
                    ob_2_class=[note_2_class(chord,ob_t) for ob_t in ob] #2 class

                    for t in range(len(ob)):
                        if k in ob_2_class[t]:
                            #Modification: multiple by how many times do k appear at timestamp t
                            total+=self.probit_at_i(t,j,ob)*ob_2_class[t].count(k)  
                            
                    #Modification: multiple by len(ob[t]), which is the total length of notes at timestamp t                    
                    self.emssion_matrix[:,k]=total/sum([self.probit_at_i(t,j,ob)*len(ob[t]) for t in range(len(ob))])  
                    
                    #smoothing
                    
                    if self.emssion_matrix[j][k]==0 :
                        self.emssion_matrix[:,k]=0.00000001
"""                        

"        \n    def train(self,obs,key_name,epochs=2):\n        #O:observed values\n        #λ:model parameters\n\n        \n        for epoch in range(epochs):\n            for ob_idx,ob in enumerate(obs):\n                self.update_state_name([key_name[ob_idx].lower()[:-1]+states for states in h_states])\n                print('running',ob_idx,key_name[ob_idx].lower()[:-1])\n                self.probit_at_i_table=np.empty((len(ob),self.no_of_state))\n                self.probit_transit_i_j_table=np.empty((len(ob),self.no_of_state,self.no_of_state))\n                self.forward_table=np.empty((len(ob),self.no_of_state))\n                self.backward_table=np.empty((len(ob),self.no_of_state))\n                self.probit_at_i_table[:]= np.NaN\n                self.probit_transit_i_j_table[:]=np.NaN\n                self.forward_table[:]= np.NaN\n                self.backward_table[:]=np.NaN\n                self.chord_probit=self.transition_matrix.sum(axis=0)\n                self.c

# Prepare Data

In [6]:
import json
with open('../data/training_data.json') as json_file:
    data = json.load(json_file)

In [7]:
#shuffle
score=np.array(list(data.keys()))
np.random.shuffle(score)
test_score=score[-1:]
score=score[:-1]
test_score,score

(array(['Nocturne_in_F_Major.mxl'], dtype='<U35'),
 array(['Prelude_in_B_Minor_Opus104a.mxl', 'Minuet_in_F.mxl',
        'Nocturne_No._20_in_C_Minor.mxl', 'Etude_in_Gb_Major_Opus_25.mxl',
        'Etude_in_Gb_Major.mxl', 'Nocturne_in_Eb_Major.mxl',
        'Il_Vecchio_Castello.mxl', 'Moonlight_Sonata_1st_Movement.mxl',
        'Nocturne_in_F_Minor.mxl', 'Etude_in_F_Minor.mxl',
        'Menuet_in_G_Minor.mxl', 'Piano_Sonata_No._11.mxl',
        'Prelude_in_A_Major.mxl', 'Prlude_Opus_28_No._4_in_E_Minor.mxl',
        'Prelude_in_B_Minor.mxl', 'notestochord.mxl',
        'Nocturne_in_E_Minor.mxl', 'Nocturne_in_C_Minor.mxl',
        'Minuet_in_G_Major_2nd.mxl', 'Etude_in_C_Minor.mxl',
        'Prelude_in_Db_Major.mxl', 'Prelude_in_G_Major.mxl',
        'Prelude_in_B_Major.mxl', 'Prelude_in_C_Minor.mxl',
        'Nocturne_in_E_Major.mxl', 'Nocturne_in_B_Major.mxl',
        'Prelude_in_F_Major.mxl', 'Etude_in_F_Major.mxl'], dtype='<U35'))

In [8]:
#prepare train data
score_name=score
x_train=[]
y_train=[]
for idx_root,name in enumerate(score_name):
    select_idx=[]

    training=[]
    training_label=[]

    training=data[name]['note_seq']
    training_label=data[name]['chord_seq']
    
    x_train.append(training)
    y_train.append(training_label)

In [9]:
#prepare test data (1 score)
test_score=test_score
score_test_choice=0

test_sample=[]
testing_label=[]
name=test_score[score_test_choice]

test_sample=data[name]['note_seq']
testing_label=data[name]['chord_seq']

test_key=[]
for e in testing_label:
    temp_key=e.split('_')[0]
    temp_name=temp_key[:-1]
    temp_key=temp_key[-1]
    if temp_key=='m':
        test_key.append(temp_name.lower()+'Minor')
    else:
        test_key.append(temp_name[0].upper()+temp_name[1:].lower()+'Major')   

# Training

In [10]:
h_states=['MinorI', 'MinorI+',
          'MinorbII', 'MinorII', 'MinorII7',
          'MinorIII',
          'MinorIV', 'MinorIV+',
          'MinorV', 'MinorV+', 'MinorV+7',
          'MinorVI', 'MinorGerVI', 'MinorFreVI', 'MinorItaVI',
          'MinorVII', 'MinorDimVII7',#, 'MinorDimVII'
          'MajorI',
          'MajorbII','MajorII','MajorII7',
          'MajorIII',
          'MajorIV',
          'MajorV','MajorV7',
          'MajorbVI','MajorGerVI','MajorFreVI','MajorItaVI','MajorVI',
          'MajorVII','MajorVII7','MajorDimVII7'
]
#all possible chordm will add the key to the chord afterward e.g. cMinorI

In [11]:
test=HMM(len(h_states),2,h_states,["outside chord","inside chord"])

In [12]:
test.train_supervisied(x_train,y_train)#initialize parameter

In [13]:
test.debug()

initial_matrix
 [5.00000000e-01 1.00000000e-08 1.00000000e-08 1.00000000e-08
 1.00000000e-08 1.00000000e-08 1.00000000e-08 1.00000000e-08
 1.00000000e-08 1.00000000e-08 1.00000000e-08 1.00000000e-08
 1.00000000e-08 1.00000000e-08 1.00000000e-08 1.00000000e-08
 1.00000000e-08 3.21428571e-01 1.00000000e-08 3.57142857e-02
 1.00000000e-08 1.00000000e-08 1.00000000e-08 1.42857143e-01
 1.00000000e-08 1.00000000e-08 1.00000000e-08 1.00000000e-08
 1.00000000e-08 1.00000000e-08 1.00000000e-08 1.00000000e-08
 1.00000000e-08]
transition_matrix
 [[1.92307692e-02 2.88461538e-02 1.44230769e-02 ... 1.00000000e-08
  1.00000000e-08 1.00000000e-08]
 [1.55555556e-01 4.44444444e-02 6.66666667e-02 ... 1.00000000e-08
  1.00000000e-08 1.00000000e-08]
 [3.33333333e-01 3.33333333e-02 1.00000000e-08 ... 1.00000000e-08
  1.00000000e-08 1.00000000e-08]
 ...
 [1.00000000e-08 1.00000000e-08 1.00000000e-08 ... 1.00000000e-08
  1.00000000e-08 1.00000000e-08]
 [1.00000000e-08 1.00000000e-08 1.00000000e-08 ... 1.000000

# Evaluation

In [14]:
import sys
sys.path.append('../modules')
import noteToChordWeighted  as noteToChord

In [15]:
len(test_key)

55

In [16]:
len(test_sample)

55

In [17]:
prediction=test.predict(test_sample,test_key)
prediction=[h_states[i] for i in prediction]

In [18]:
#check prediction and label side by side
for i in range(len(prediction)):
    print(prediction[i],testing_label[i])

MajorI F#M_V
MajorV F#M_I
MajorI F#M_V
MajorV F#M_I
MinorV+7 G#m_V+
MinorI G#m_I
MajorII F#M_II
MajorV F#M_I
MajorI F#M_IV
MajorV F#M_V
MajorI F#M_VI
MajorV F#M_V
MajorI F#M_I
MajorV F#M_V
MajorI F#M_I
MajorV F#M_V
MajorI F#M_I
MinorV+7 G#m_V+
MinorI G#m_I
MajorV F#M_I
MajorI F#M_IV
MajorV F#M_V
MajorI F#M_I
MinorV+7 F#m_V+
MinorI F#m_I
MinorGerVI F#m_GermanVI
MinorI F#m_IV+
MinorI A#M_I
MinorI F#M_V
MajorDimVII7 FM_Dim7
MajorV7 AM_V
MinorV+7 F#m_V+7
MinorI F#m_I
MinorV+7 F#m_V+7
MinorVI F#m_VI
MinorV+ F#m_V
MinorIV F#m_II
MinorV+7 F#m_V+
MinorI F#m_I
MinorV+7 F#m_V+
MajorV F#M_V
MajorI F#M_I
MajorV F#M_V
MajorI F#M_I
MinorV+7 G#m_V+
MinorI G#m_I
MajorII F#M_II
MajorV F#M_V
MajorI F#M_IV
MajorV F#M_V
MajorI F#M_I
MajorV F#M_V
MajorI F#M_VI
MajorV F#M_V
MajorI F#M_I


In [19]:
for idx,e in enumerate(testing_label):
    if 'M' in e:
        testing_label[idx]=testing_label[idx].replace("M_", "Major")
        testing_label[idx]=testing_label[idx][0].upper()+testing_label[idx][1:]
    else:
        testing_label[idx]=testing_label[idx].replace("m_", "Minor")
        testing_label[idx]=testing_label[idx][0].lower()+testing_label[idx][1:]

In [20]:
#compare with chord2note
note2chord_ans=[]
for notes,key in zip(test_sample,test_key):
    note2chord_ans.append(noteToChord.NoteToChord(notes,key,1,0)[0]['Chord'])
    
for idx,e in enumerate(testing_label):
    testing_label[idx]='M'+e.split('M')[1]
for idx,e in enumerate(note2chord_ans):
    note2chord_ans[idx]='M'+e.split('M')[1]
#evaluation
correct=0
wrong=0
for idx,label in enumerate(note2chord_ans):
    
    if label.replace('7','').replace('German','Ger').replace('Gb','F#').upper()==testing_label[idx].replace('7','').replace('German','Ger').upper():
        correct+=1
    else:
        wrong+=1
        print(label,testing_label[idx])
print('acc',correct/(correct+wrong),correct,wrong)   

MajorVI MajorI
MajorII7 MajorV
MajorVII MajorV
MajorDimVII7 MajorDim7
MinorV+ MinorV
MajorIV MajorV
MajorI MajorV
acc 0.8727272727272727 48 7


In [21]:
#compare with 2-HMM
#evaluation
correct=0
wrong=0
for idx,label in enumerate(prediction):
    if label.replace('7','').replace('German','Ger').upper()==testing_label[idx].replace('7','').replace('German','Ger').upper():
        correct+=1
    else:
        wrong+=1
        print(label,testing_label[idx])
print('acc',correct/(correct+wrong),correct,wrong)

MajorI MajorV
MajorV MajorI
MajorI MajorV
MajorV MajorI
MajorV MajorI
MajorI MajorIV
MajorI MajorVI
MajorV MajorI
MajorI MajorIV
MinorI MinorIV+
MinorI MajorI
MinorI MajorV
MajorDimVII7 MajorDim7
MinorV+ MinorV
MinorIV MinorII
MajorI MajorIV
MajorI MajorVI
acc 0.6909090909090909 38 17
